In [73]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [191]:
import torch 
import tabpfn

In [192]:
import random
import time
import warnings
from datetime import datetime

import torch

import numpy as np

import matplotlib.pyplot as plt
#from tabpfn.scripts.differentiable_pfn_evaluation import eval_model_range
from tabpfn.scripts.model_builder import get_model, get_default_spec, save_model, load_model
from tabpfn.scripts.transformer_prediction_interface import transformer_predict, get_params_from_config, load_model_workflow

from tabpfn.scripts.model_configs import *

from tabpfn.datasets import load_openml_list, open_cc_dids, open_cc_valid_dids
#from tabpfn.priors.utils import plot_prior, plot_features
#from tabpfn.priors.utils import uniform_int_sampler_f

from tabpfn.scripts.tabular_metrics import calculate_score_per_method, calculate_score
#from tabpfn.scripts.tabular_evaluation import evaluate

#from tabpfn.priors.differentiable_prior import DifferentiableHyperparameterList, draw_random_style, merge_style_with_info
#from scripts import tabular_metrics
#from tabpfntabpfn.otebook_utils import *

In [193]:
large_datasets = True
max_samples = 10000 if large_datasets else 5000
bptt = 10000 if large_datasets else 3000
suite='cc'

In [194]:
device = 'cpu'
base_path = '.'
max_features = 10 # P

In [214]:
def reload_config(config_type='causal', task_type='multiclass', longer=0):
    config = get_prior_config(config_type=config_type)
    
    config['prior_type'], config['differentiable'], config['flexible'] = 'mlp', True, True
    
    model_string = ''
    
    config['epochs'] = 12000
    config['recompute_attn'] = True

    config['max_num_classes'] = 10
    config['num_classes'] = uniform_int_sampler_f(2, config['max_num_classes'])
    config['balanced'] = False
    model_string = model_string + '_multiclass'
    
    model_string = model_string + '_'+datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    
    return config, model_string

In [ ]:
config, model_string = reload_config(longer=1)

config['bptt_extra_samples'] = None

# diff
config['output_multiclass_ordered_p'] = 0.
del config['differentiable_hyperparameters']['output_multiclass_ordered_p']

config['multiclass_type'] = 'rank'
del config['differentiable_hyperparameters']['multiclass_type']

config['sampling'] = 'normal' # vielleicht schlecht?
del config['differentiable_hyperparameters']['sampling']

config['pre_sample_causes'] = True
# end diff

config['multiclass_loss_type'] = 'nono' # 'compatible'
config['normalize_to_ranking'] = False # False

config['categorical_feature_p'] = .2 # diff: .0

# turn this back on in a random search!?
config['nan_prob_no_reason'] = .0
config['nan_prob_unknown_reason'] = .0 # diff: .0
config['set_value_to_nan'] = .1 # diff: 1.

config['normalize_with_sqrt'] = False

config['new_mlp_per_example'] = True
config['prior_mlp_scale_weights_sqrt'] = True
config['batch_size_per_gp_sample'] = None

config['normalize_ignore_label_too'] = False

config['differentiable_hps_as_style'] = False
config['max_eval_pos'] = 1000

config['random_feature_rotation'] = True
config['rotate_normalized_labels'] = True

config["mix_activations"] = False # False heisst eig True

config['emsize'] = 512
config['nhead'] = config['emsize'] // 128
config['bptt'] = 50 # N
config['canonical_y_encoder'] = False

    
config['aggregate_k_gradients'] = 8
config['batch_size'] = 8*config['aggregate_k_gradients']
config['num_steps'] = int(1e10)
config['epochs'] = int(1e10)
config['total_available_time_in_s'] = None #60*60*22 # 22 hours for some safety...

config['train_mixed_precision'] = True
config['efficient_eval_masking'] = True

#config["num_features"] = 10 # P

config_sample = evaluate_hypers(config)

In [223]:
config["num_features"]

100

In [224]:
config_sample["num_features_used"] = {'uniform_int_sampler_f(3,max_features)': uniform_int_sampler_f(10, 10)}
config_sample["num_features"] = 10

In [233]:
config_sample['batch_size'] = 4
model = get_model(config_sample, device, should_train=False, verbose=-10) # , state_dict=model[2].state_dict()
(hp_embedding, data, _), targets, single_eval_pos = next(iter(model[3]))

Using style prior: True
Using cpu:0 device
Using a Transformer with 25.78 M parameters


In [234]:
data_res = []
for i in range(100):
    (hp_embedding, data, _), targets, single_eval_pos = next(iter(model[3]))
    data_res.append(data.squeeze().cpu())


In [232]:
data_res[0].shape

torch.Size([50, 10])

In [212]:
torch.sum(data[0,0] != 0)

tensor(10)

In [213]:
data.shape

torch.Size([50, 1, 10])